<a href="https://colab.research.google.com/github/nmq443/cognitive-science-final-project/blob/quang-branch/torcheeg_atcnet-with_preprocessed_data_cwt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# if run on colab
"""
!pip install torcheeg
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
"""

"\n!pip install torcheeg\nfrom google.colab import drive\ndrive.mount('/content/drive', force_remount=True)\n"

In [1]:
# if run on kaggle
!pip install torcheeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.0 MB/s eta 0:00:00
  Created wheel for torcheeg: filename=t

In [2]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

In [3]:
# if run on kaggle
root_data_path = '/kaggle/input/BCICIV-2a-mat'

# if run on colab
# root_data_path = '/content/drive/MyDrive/BCICIV-2a-mat'

# if run on local machine
# root_data_path = './BCICIV-2a-mat/'

In [4]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    online_transform=transforms.Compose([
        transforms.To2d(),
        transforms.MinMaxNormalize(),
        transforms.ToTensor(),
        transforms.RandomNoise(p=0.3),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-22 08:01:30] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./examples_pipeline/bciciv-2a.
[2024-05-22 08:01:30] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/18 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()

[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 1it [00:01,  1.15s/it]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 12it [00:01, 12.83it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 23it [00:01, 25.97it/s]
[RECORD /kaggle/input/BCICIV-2a-mat/A04T.mat]: 33it [00:01, 37.91it/

In [5]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 
      start_at  end_at   clip_id subject_id  trial_id session subject  run  \
0          251    2001    A04T_0        A04         0       T     A04    1   
1         2254    4004    A04T_1        A04         1       T     A04    1   
2         4172    5922    A04T_2        A04         2       T     A04    1   
3         6124    7874    A04T_3        A04         3       T     A04    1   
4         8132    9882    A04T_4        A04         4       T     A04    1   
...        ...     ...       ...        ...       ...     ...     ...  ...   
5179     86751   88501  A04E_283        A04        43       E     A04    8   
5180     88657   90407  A04E_284        A04        44       E     A04    8   
5181     90585   92335  A04E_285        A04        45       E     A04    8   
5182     92699   94449  A04E_286        A04        46       E     A04    8   
5183     94758   96508  A04E_287        A04        47       E     A04    8   

      label  _record_id  
0         4   _recor

In [6]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [7]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=8
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=8
    )

    model = ATCNet(
        num_classes=4,
        num_electrodes=22,
        in_channels=1,
        chunk_size=7*250,
    )

    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader,
        max_epochs=50,
        default_root_dir=f'./examples_pipeline/atcnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-22 08:02:23] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-05-22 08:02:23] INFO (torcheeg/MainThread) 😊 | Please set split_path to ./examples_pipeline/split for the next run, if you want to use the same setting for the experiment.
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Conv

Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


Testing: |          | 0/? [00:00<?, ?it/s]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
!mkdir weights

In [ ]:
atc_weights_path = './weights/atc_weights.pt'
torch.save(model.state_dict(), atc_weights_path)
# to load weight:
# model.load_state_dict(torch.load(atc_weights_path))